codice ottimizzato e definitivo, con:

Timeout globale per richieste a Wikipedia

Riduzione delle varianti per pagina Wikipedia

Salvataggio intermedio ogni 20 risultati

Cache in memoria (con opzione per persistente su file)

Log chiari e dettagliati

Gestione sicura degli errori per evitare blocchi

In [1]:
import pandas as pd
import wikipedia
import re
import concurrent.futures
from tqdm import tqdm
import socket
import time
import os

# Timeout globale per tutte le richieste socket (Wikipedia compresa)
socket.setdefaulttimeout(10)

# Impostazioni iniziali
wikipedia.set_lang("it")
wikipedia.set_rate_limiting(True)

file_path = "File Unico Sanremo.xlsx"
output_file = "Artisti_con_data_nascita_ottimizzata.xlsx"
temp_file = "TEMP_output.xlsx"
cache_file = "dob_cache.pkl"

# Carica dati
df = pd.read_excel(file_path, sheet_name="Artisti Unici con genere")
df_individuali = df[df["Genere"].isin(["Maschio", "Femmina"])].copy()
df_gruppi = df[~df["Genere"].isin(["Maschio", "Femmina"])].copy()
df_gruppi["Data di nascita"] = "Da verificare"

# Dizionario mesi
mesi_ita = {
    "gennaio": "01", "febbraio": "02", "marzo": "03", "aprile": "04",
    "maggio": "05", "giugno": "06", "luglio": "07", "agosto": "08",
    "settembre": "09", "ottobre": "10", "novembre": "11", "dicembre": "12"
}

# Cache su file
try:
    dob_cache = pd.read_pickle(cache_file)
except:
    dob_cache = {}

# Funzione per cercare pagina Wikipedia con meno varianti
def cerca_pagina(artista):
    varianti = [
        artista,
        f"{artista} (cantante)",
        f"{artista} (rapper)",
        f"{artista} (musicista)",
        f"{artista} cantante"
    ]
    for nome in varianti:
        try:
            return wikipedia.page(nome, auto_suggest=False, preload=False)
        except (wikipedia.exceptions.DisambiguationError, wikipedia.exceptions.PageError):
            continue
        except Exception:
            continue
    return None

# Estrai data da testo
def estrai_data_da_testo(testo):
    testo = testo.lower()

    match = re.search(r'\(([^,]+),\s*(\d{1,2})\s+([a-z]+)\s+(\d{4})\)', testo)
    if match:
        giorno = match.group(2).zfill(2)
        mese = mesi_ita.get(match.group(3))
        anno = match.group(4)
        if mese:
            return f"{giorno}/{mese}/{anno}"

    match = re.search(r'(nato|nata) il (\d{1,2})\s+([a-z]+)\s+(\d{4})', testo)
    if match:
        giorno = match.group(2).zfill(2)
        mese = mesi_ita.get(match.group(3))
        anno = match.group(4)
        if mese:
            return f"{giorno}/{mese}/{anno}"

    match = re.search(r'(nato|nata) a [a-zàèéòùì\s]+ nel (\d{4})', testo)
    if match:
        anno = match.group(2)
        return f"01/01/{anno}"

    return None

# Funzione principale
def estrai_data_nascita(artista):
    if artista in dob_cache:
        return dob_cache[artista]

    if any(sep in artista.lower() for sep in [",", " e ", " - "]):
        print(f"⚠️ Nomi multipli rilevati, salto: {artista}")
        return "Da verificare"

    print(f"🟨 Elaborazione: {artista}")
    pagina = cerca_pagina(artista)
    if pagina is None:
        print(f"❌ Nessuna pagina trovata per: {artista}")
        return "Da verificare"

    contenuto = (pagina.summary + " " + pagina.content).lower()

    # Pseudonimo
    if "pseudonimo di" in contenuto:
        match_nome = re.search(r"pseudonimo di ([a-zàèéòùì'\s]+)", contenuto)
        if match_nome:
            vero_nome = match_nome.group(1).strip()
            print(f"🔍 Pseudonimo rilevato, provo con vero nome: {vero_nome}")
            pagina_vero_nome = cerca_pagina(vero_nome)
            if pagina_vero_nome:
                contenuto += " " + pagina_vero_nome.summary.lower()

    data = estrai_data_da_testo(contenuto)
    if data:
        dob_cache[artista] = data
        return data

    print(f"⚠️ Nessuna data trovata per: {artista}")
    return "Da verificare"

# Esecuzione parallela con salvataggio intermedio
def apply_parallel(df, func, descrizione):
    results = []
    processed = 0

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(func, artista): artista for artista in df["Artista"]}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc=descrizione):
            artista = futures[future]
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                print(f"❌ Errore durante {descrizione} su: {artista} — {e}")
                results.append("Errore")

            processed += 1

            # Salvataggio intermedio ogni 20 artisti
            if processed % 20 == 0 or processed == len(futures):
                df["Data di nascita"] = results + [""] * (len(df) - len(results))
                df_finale_temp = pd.concat([df, df_gruppi])
                df_finale_temp.to_excel(temp_file, index=False)
                pd.to_pickle(dob_cache, cache_file)
                print(f"💾 Salvataggio temporaneo completato ({processed}/{len(futures)})")

    return results

# Applica estrazione e salva file finale
df_individuali["Data di nascita"] = apply_parallel(df_individuali, estrai_data_nascita, "Estrazione data nascita")
df_finale = pd.concat([df_individuali, df_gruppi]).sort_index()
df_finale.to_excel(output_file, index=False)
pd.to_pickle(dob_cache, cache_file)

print(f"\n✅ File finale salvato con successo: {output_file}")


🟨 Elaborazione: Achille Togliani
🟨 Elaborazione: Oscar Carboni
🟨 Elaborazione: Gino Latilla
🟨 Elaborazione: Zucchero
🟨 Elaborazione: Will
🟨 Elaborazione: Virginio
🟨 Elaborazione: Ultimo
🟨 Elaborazione: Tricarico
🟨 Elaborazione: Tiziano Orecchio
🟨 Elaborazione: Stefano Tosi


Estrazione data nascita:   0%|          | 0/470 [00:00<?, ?it/s]D:\bocci\anaconda3\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file D:\bocci\anaconda3\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
Estrazione data nascita:   0%|          | 1/470 [00:01<13:30,  1.73s/it]

⚠️ Nessuna data trovata per: Virginio
🟨 Elaborazione: Ugo Molinari
⚠️ Nessuna data trovata per: Gino Latilla
🟨 Elaborazione: Gianni Marzocchi
⚠️ Nessuna data trovata per: Oscar Carboni
🟨 Elaborazione: Stefano Rosso
⚠️ Nessuna data trovata per: Achille Togliani
🟨 Elaborazione: Silvano Vittorio


Estrazione data nascita:   1%|          | 5/470 [00:02<02:28,  3.12it/s]

⚠️ Nessuna data trovata per: Tricarico
🟨 Elaborazione: Sangiovanni


...

Estrazione data nascita:  99%|█████████▊| 463/470 [02:14<00:02,  3.02it/s]

🔍 Pseudonimo rilevato, provo con vero nome: carla bissi
🔍 Pseudonimo rilevato, provo con vero nome: sara mattei
🔍 Pseudonimo rilevato, provo con vero nome: gaia gozzi


Estrazione data nascita: 100%|██████████| 470/470 [02:18<00:00,  3.39it/s]

💾 Salvataggio temporaneo completato (470/470)



✅ File finale salvato con successo: Artisti_con_data_nascita_ottimizzata.xlsx
